In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
dataset = pd.read_csv('/content/sample_data/exoplanets_2018.csv') 
#dataset = dataset.rename(columns=header_list)
dataset.head()

,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_teq_err1,koi_teq_err2,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,9.488036,2.780000e-05,-2.780000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,616.0,19.5,-19.5,2.26,0.26,-0.15,793.0,NaN,NaN,93.59,29.45,-16.65,35.8,1.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,54.418383,2.480000e-04,-2.480000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,875.0,35.5,-35.5,2.83,0.32,-0.19,443.0,NaN,NaN,9.11,2.87,-1.62,25.8,2.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,K00753.01,NaN,CANDIDATE,CANDIDATE,0.000,0,0,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10800.0,171.0,-171.0,14.60,3.92,-1.31,638.0,NaN,NaN,39.30,31.04,-10.49,76.3,1.0,q1_q17_dr25_tce,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8080.0,12.8,-12.8,33.46,8.50,-2.83,1395.0,NaN,NaN,891.96,668.95,-230.35,505.6,1.0,q1_q17_dr25_tce,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.0,16.9,-16.9,2.75,0.88,-0.35,1406.0,NaN,NaN,926.16,874.33,-314.24,40.9,1.0,q1_q17_dr25_tce,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [3]:
pd.set_option("display.max_columns", None)

In [4]:
dataset=dataset[['koi_disposition','koi_fpflag_nt','koi_fpflag_ss','koi_fpflag_co','koi_fpflag_ec','koi_depth', 'koi_model_snr']]

In [5]:
for col in dataset.columns:
    print(col)

koi_disposition
koi_fpflag_nt
koi_fpflag_ss
koi_fpflag_co
koi_fpflag_ec
koi_depth
koi_model_snr


In [6]:
dataset["koi_disposition"].replace({"CONFIRMED": "1", "CANDIDATE": "0", "FALSE POSITIVE": "0"}, inplace=True)
dataset.isnull().sum().sum()
dataset=dataset.dropna()  
dataset["koi_disposition"].value_counts()

0    6845
1    2356
Name: koi_disposition, dtype: int64

In [ ]:
dataset.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_depth,koi_model_snr
0,1,0,0,0,0,616.0,35.8
1,1,0,0,0,0,875.0,25.8
2,0,0,0,0,0,10800.0,76.3
3,0,0,1,0,0,8080.0,505.6
4,1,0,0,0,0,603.0,40.9


In [7]:
dataset = dataset.to_numpy()

# Sepparating the label (Y) from the input features (X)
Y = dataset[:, 0]
Y = np.array(Y, dtype=int)
X = dataset[:, 1:]

# Sanity check
print(X.shape, Y.shape)

(9201, 6) (9201,)


In [ ]:
#@title Hidden
# Separate data with label 0 and label 1

x_0 = X[Y == 0, :]
x_1 = X[Y == 1, :]
y_0 = Y[Y==0]
y_1 = Y[Y==1]

# Sanity check
print(x_0.shape, y_0.shape)
print(x_1.shape, y_1.shape)

In [8]:
# Split 70% of the data for training set and 30% for testing set

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=2001, shuffle=True, stratify=Y)
# x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(x_1, y_1, test_size=0.3, random_state=2001,shuffle=True, stratify=Y)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(6440, 6) (6440,)
(2761, 6) (2761,)


In [ ]:
#@title Hidden
num_sample = 1500 # sample per class, total = 5000

# Take the first 5000 samples (1500 from each class) from training set for X_train and Y_train
X_train = np.concatenate((x_train_0[:num_sample, :], x_train_1[:num_sample, :]), axis=0)
Y_train = np.concatenate((y_train_0[:num_sample], y_train_1[:num_sample]), axis=0)

# Take the first 5000 samples (1500 from each class) from testing set for X_test and Y_test
X_test = np.concatenate((x_test_0[:num_sample, :], x_test_1[:num_sample, :]), axis=0)
Y_test = np.concatenate((y_test_0[:num_sample], y_test_1[:num_sample]), axis=0)

# Manual Check 
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

In [9]:
#Save extracted datasets on the disk
#!mkdir 'KeplerDataset'

np.savetxt('/content/sample_data/X_train_6694.txt', X_train)
np.savetxt('/content/sample_data/X_test_2870.txt', X_test)
np.savetxt('/content/sample_data/Y_train_6694.txt', Y_train)
np.savetxt('/content/sample_data/Y_test_2870.txt', Y_test)

In [10]:
#@title Hidden
# Code to load the saved subset


X_train = np.loadtxt('/content/sample_data/X_train_6694.txt')
Y_train = np.loadtxt('/content/sample_data/Y_train_6694.txt')

X_test = np.loadtxt('/content/sample_data/X_test_2870.txt')
Y_test = np.loadtxt('/content/sample_data/Y_test_2870.txt')

# Check
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(6440, 6) (6440,)
(2761, 6) (2761,)


In [31]:
#Training the Quantum Circuit in Simulator
#pip install pennylane
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer

import tensorflow as tf
from tensorflow.keras.utils import to_categorical


In [32]:
# Set a random seed
np.random.seed(42)

In [33]:
# Define output labels as quantum state vectors
label_0 = [[1], [0]]
label_1 = [[0], [1]]


def density_matrix(state):
    """Calculates the density matrix representation of a state.

    Args:
        state (array[complex]): array representing a quantum state vector

    Returns:
        dm: (array[complex]): array representing the density matrix
    """
    return np.outer(state,  np.conj(state))

state_labels = [label_0, label_1]
dm_labels = [density_matrix(state_labels[i]) for i in range(2)]

In [34]:
from keras import backend as K


# Alpha Custom Layer
class class_weights(tf.keras.layers.Layer):
    def __init__(self):
        super(class_weights, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(1, 2), dtype="float32"),
            trainable=True,
        )

    def call(self, inputs):
        return (inputs * self.w)

In [35]:
from keras import backend as K


# Alpha Custom Layer
class class_weights(tf.keras.layers.Layer):
    def __init__(self):
        super(class_weights, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(1, 2), dtype="float32"),
            trainable=True,
        )

    def call(self, inputs):
        return (inputs * self.w)

In [36]:
num_fc_layer = 5 # number of layer
params_fix = np.random.uniform(size=(2, num_fc_layer, 6))
print(params_fix)

[[[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452]
  [0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985]
  [0.83244264 0.21233911 0.18182497 0.18340451 0.30424224 0.52475643]
  [0.43194502 0.29122914 0.61185289 0.13949386 0.29214465 0.36636184]
  [0.45606998 0.78517596 0.19967378 0.51423444 0.59241457 0.04645041]]

 [[0.60754485 0.17052412 0.06505159 0.94888554 0.96563203 0.80839735]
  [0.30461377 0.09767211 0.68423303 0.44015249 0.12203823 0.49517691]
  [0.03438852 0.9093204  0.25877998 0.66252228 0.31171108 0.52006802]
  [0.54671028 0.18485446 0.96958463 0.77513282 0.93949894 0.89482735]
  [0.59789998 0.92187424 0.0884925  0.19598286 0.04522729 0.32533033]]]


In [37]:
n_qubits = 2  # number of class
dev_fc = qml.device("default.qubit", wires=n_qubits)
layer_id = 0 # the layer index to be trained, it starts from zero because of Numpy convention

@qml.qnode(dev_fc)
def q_fc(params, inputs):
    """A variational quantum circuit representing the DRC.

    Args:
        params (array[float]): array of parameters
        inputs (array[float]): 1-d input vector (data sample)

    Returns:
        array[float]: 1-d output vector in the form of [alpha0*<O0>, alpha1*<O1>]
    """
    
    # layer iteration
    for l in range(len(params_fix[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
              if l == layer_id: # train only the specified layer
                qml.Rot(*(params[0][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][3*g:3*(g+1)]), wires=q)
              else:
                qml.Rot(*(params_fix[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params_fix[1][l][3*g:3*(g+1)]), wires=q)
    
    return [qml.expval(qml.Hermitian(dm_labels[i], wires=[i])) for i in range(n_qubits)]

In [38]:
n_component = 6 # number of features used

X = tf.keras.Input(shape=(n_component,), name='Input_Layer')


# Quantum Layer
q_fc_layer = qml.qnn.KerasLayer(q_fc, {"params": (2, n_component)}, output_dim=2)(X)

# Alpha Layer
alpha_layer = class_weights()(q_fc_layer)

model = tf.keras.Model(inputs=X, outputs=alpha_layer)

In [39]:
opt = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(opt, loss='mse', metrics=["accuracy"])

In [40]:
filepath = "/content/sample_data/Best6_layer5(layer_id=0)_set1000_8epoch_saved-model-{epoch:02d}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1,
                                                save_weights_only=True, save_best_only=False, mode='auto')

In [42]:
H = model.fit(X_train, to_categorical(Y_train), epochs=1, batch_size=256, initial_epoch=0,
              validation_data=(X_test, to_categorical(Y_test)), verbose=1,
              callbacks=[checkpoint])

26/26 [==============================] - 614s 24s/step - loss: 0.3244 - accuracy: 0.5705 - val_loss: 0.2707 - val_accuracy: 0.6121

Epoch 00001: saving model to /content/sample_data/Best6_layer5(layer_id=0)_set1000_8epoch_saved-model-01.hdf5


In [ ]:
!pwd
!ls /content/sample_data/Best

/content
ls: cannot access '/content/sample_data/Best': No such file or directory


In [43]:
model.load_weights('/content/sample_data/Best6_layer5(layer_id=0)_set1000_8epoch_saved-model-03.hdf5')

# keep that parameters and replace the initial one
params_layer_0 = model.get_weights()[0]
params_fix[:, 0, :] = params_layer_0

**2nd Layer**

In [44]:
n_qubits = 2  # number of class
dev_fc = qml.device("default.qubit", wires=n_qubits)
layer_id = 1 # the layer index to be trained, it starts from zero because of Numpy convention

@qml.qnode(dev_fc)
def q_fc(params, inputs):
    """A variational quantum circuit representing the DRC.

    Args:
        params (array[float]): array of parameters
        inputs (array[float]): 1-d input vector (data sample)

    Returns:
        array[float]: 1-d output vector in the form of [alpha0*<O0>, alpha1*<O1>]
    """
    
    # layer iteration
    for l in range(len(params_fix[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
              if l == layer_id: # train only the specified layer
                qml.Rot(*(params[0][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][3*g:3*(g+1)]), wires=q)
              else:
                qml.Rot(*(params_fix[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params_fix[1][l][3*g:3*(g+1)]), wires=q)
    
    return [qml.expval(qml.Hermitian(dm_labels[i], wires=[i])) for i in range(n_qubits)]

In [45]:
n_component = 6 # number of features used

X = tf.keras.Input(shape=(n_component,), name='Input_Layer')


# Quantum Layer
q_fc_layer = qml.qnn.KerasLayer(q_fc, {"params": (2, n_component)}, output_dim=2)(X)

# Alpha Layer
alpha_layer = class_weights()(q_fc_layer)

model = tf.keras.Model(inputs=X, outputs=alpha_layer)

In [46]:
opt = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(opt, loss='mse', metrics=["accuracy"])

In [47]:
# Note: please put the correct directory to save the weights accordingly if you want to run the code
filepath = "/content/sample_data/Best6_layer5(layer_id=1)_3epoch_saved-model-{epoch:02d}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1,
                                                save_weights_only=True, save_best_only=False, mode='auto')

In [49]:
H = model.fit(X_train, to_categorical(Y_train), epochs=1, batch_size=256, initial_epoch=0,
              validation_data=(X_test, to_categorical(Y_test)), verbose=1,
              callbacks=[checkpoint])

26/26 [==============================] - 607s 24s/step - loss: 0.3310 - accuracy: 0.5741 - val_loss: 0.2672 - val_accuracy: 0.6096

Epoch 00001: saving model to /content/sample_data/Best6_layer5(layer_id=1)_3epoch_saved-model-01.hdf5


In [50]:
# load the parameters that is considered giving the best balance between train and test accuracy
model.load_weights('/content/sample_data/Best6_layer5(layer_id=1)_3epoch_saved-model-02.hdf5')


# keep that parameters and replace the initial one
params_layer_1 = model.get_weights()[0]
params_fix[:, 1, :] = params_layer_1

**3rd Layer**

In [51]:
n_qubits = 2  # number of class
dev_fc = qml.device("default.qubit", wires=n_qubits)
layer_id = 2 # the layer index to be trained, it starts from zero because of Numpy convention

@qml.qnode(dev_fc)
def q_fc(params, inputs):
    """A variational quantum circuit representing the DRC.

    Args:
        params (array[float]): array of parameters
        inputs (array[float]): 1-d input vector (data sample)

    Returns:
        array[float]: 1-d output vector in the form of [alpha0*<O0>, alpha1*<O1>]
    """
    
    # layer iteration
    for l in range(len(params_fix[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
              if l == layer_id: # train only the specified layer
                qml.Rot(*(params[0][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][3*g:3*(g+1)]), wires=q)
              else:
                qml.Rot(*(params_fix[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params_fix[1][l][3*g:3*(g+1)]), wires=q)
    
    return [qml.expval(qml.Hermitian(dm_labels[i], wires=[i])) for i in range(n_qubits)]

In [52]:
n_component = 6 # number of features used

X = tf.keras.Input(shape=(n_component,), name='Input_Layer')


# Quantum Layer
q_fc_layer = qml.qnn.KerasLayer(q_fc, {"params": (2, n_component)}, output_dim=2)(X)

# Alpha Layer
alpha_layer = class_weights()(q_fc_layer)

model = tf.keras.Model(inputs=X, outputs=alpha_layer)

In [53]:
opt = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(opt, loss='mse', metrics=["accuracy"])

In [56]:
# Note: please put the correct directory to save the weights accordingly if you want to run the code
filepath = "/content/sample_data/Best6_layer5(layer_id=2)_3epoch_saved-model-{epoch:02d}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1,
                                                save_weights_only=True, save_best_only=False, mode='auto')

In [57]:
H = model.fit(X_train, to_categorical(Y_train), epochs=3, batch_size=256, initial_epoch=0,
              validation_data=(X_test, to_categorical(Y_test)), verbose=1,
              callbacks=[checkpoint])

Epoch 1/3
26/26 [==============================] - 606s 23s/step - loss: 0.2707 - accuracy: 0.6230 - val_loss: 0.2717 - val_accuracy: 0.6186

Epoch 00001: saving model to /content/sample_data/Best6_layer5(layer_id=2)_3epoch_saved-model-01.hdf5
Epoch 2/3
26/26 [==============================] - 605s 23s/step - loss: 0.2704 - accuracy: 0.6132 - val_loss: 0.2709 - val_accuracy: 0.6222

Epoch 00002: saving model to /content/sample_data/Best6_layer5(layer_id=2)_3epoch_saved-model-02.hdf5
Epoch 3/3
26/26 [==============================] - 610s 24s/step - loss: 0.2675 - accuracy: 0.6219 - val_loss: 0.2729 - val_accuracy: 0.6190

Epoch 00003: saving model to /content/sample_data/Best6_layer5(layer_id=2)_3epoch_saved-model-03.hdf5


In [ ]:
# load the parameters that is considered giving the best balance between train and test accuracy
model.load_weights('./Model/Best6_layer5(layer_id=2)_set5000_10epoch_saved-model-08.hdf5')


# keep that parameters and replace the initial one
params_layer_2 = model.get_weights()[0]
params_fix[:, 2, :] = params_layer_2

**4th Layer**

In [ ]:
n_qubits = 2  # number of class
dev_fc = qml.device("default.qubit", wires=n_qubits)
layer_id = 3 # the layer index to be trained, it starts from zero because of Numpy convention

@qml.qnode(dev_fc)
def q_fc(params, inputs):
    """A variational quantum circuit representing the DRC.

    Args:
        params (array[float]): array of parameters
        inputs (array[float]): 1-d input vector (data sample)

    Returns:
        array[float]: 1-d output vector in the form of [alpha0*<O0>, alpha1*<O1>]
    """
    
    # layer iteration
    for l in range(len(params_fix[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
              if l == layer_id: # train only the specified layer
                qml.Rot(*(params[0][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][3*g:3*(g+1)]), wires=q)
              else:
                qml.Rot(*(params_fix[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params_fix[1][l][3*g:3*(g+1)]), wires=q)
    
    return [qml.expval(qml.Hermitian(dm_labels[i], wires=[i])) for i in range(n_qubits)]

In [ ]:
n_component = 6 # number of features used

X = tf.keras.Input(shape=(n_component,), name='Input_Layer')


# Quantum Layer
q_fc_layer = qml.qnn.KerasLayer(q_fc, {"params": (2, n_component)}, output_dim=2)(X)

# Alpha Layer
alpha_layer = class_weights()(q_fc_layer)

model = tf.keras.Model(inputs=X, outputs=alpha_layer)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(opt, loss='mse', metrics=["accuracy"])

In [ ]:
# Note: please put the correct directory to save the weights accordingly if you want to run the code
filepath = "./Model/Best6_layer5(layer_id=3)_set5000_10epoch_saved-model-{epoch:02d}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1,
                                                save_weights_only=True, save_best_only=False, mode='auto')

In [ ]:
H = model.fit(X_train, to_categorical(Y_train), epochs=10, batch_size=128, initial_epoch=0,
              validation_data=(X_test, to_categorical(Y_test)), verbose=1,
              callbacks=[checkpoint])

In [ ]:
# load the parameters that is considered giving the best balance between train and test accuracy
model.load_weights('./Model/Best6_layer5(layer_id=3)_set5000_10epoch_saved-model-08.hdf5')


# keep that parameters and replace the initial one
params_layer_3 = model.get_weights()[0]
params_fix[:, 3, :] = params_layer_3



**5th Layer**

In [ ]:
n_qubits = 2  # number of class
dev_fc = qml.device("default.qubit", wires=n_qubits)
layer_id = 4 # the layer index to be trained, it starts from zero because of Numpy convention

@qml.qnode(dev_fc)
def q_fc(params, inputs):
    """A variational quantum circuit representing the DRC.

    Args:
        params (array[float]): array of parameters
        inputs (array[float]): 1-d input vector (data sample)

    Returns:
        array[float]: 1-d output vector in the form of [alpha0*<O0>, alpha1*<O1>]
    """
    
    # layer iteration
    for l in range(len(params_fix[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
              if l == layer_id: # train only the specified layer
                qml.Rot(*(params[0][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][3*g:3*(g+1)]), wires=q)
              else:
                qml.Rot(*(params_fix[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params_fix[1][l][3*g:3*(g+1)]), wires=q)
    
    return [qml.expval(qml.Hermitian(dm_labels[i], wires=[i])) for i in range(n_qubits)]

In [ ]:
n_component = 6 # number of features used

X = tf.keras.Input(shape=(n_component,), name='Input_Layer')


# Quantum Layer
q_fc_layer = qml.qnn.KerasLayer(q_fc, {"params": (2, n_component)}, output_dim=2)(X)

# Alpha Layer
alpha_layer = class_weights()(q_fc_layer)

model = tf.keras.Model(inputs=X, outputs=alpha_layer)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(opt, loss='mse', metrics=["accuracy"])

In [ ]:
# Note: please put the correct directory to save the weights accordingly if you want to run the code
filepath = "./Model/Best6_layer5(layer_id=4)_set5000_10epoch_saved-model-{epoch:02d}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1,
                                                save_weights_only=True, save_best_only=False, mode='auto')

In [ ]:
H = model.fit(X_train, to_categorical(Y_train), epochs=10, batch_size=128, initial_epoch=0,
              validation_data=(X_test, to_categorical(Y_test)), verbose=1,
              callbacks=[checkpoint])

In [ ]:
# load the parameters that is considered giving the best balance between train and test accuracy
model.load_weights('./Model/Best6_layer5(layer_id=4)_set5000_10epoch_saved-model-08.hdf5')


# keep that parameters and replace the initial one
params_layer_4 = model.get_weights()[0]
params_fix[:, 4, :] = params_layer_4

**Final Training**

In [ ]:
n_qubits = 2  # number of class
dev_fc = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev_fc)
def q_fc(params, inputs):
    """A variational quantum circuit representing the DRC.

    Args:
        params (array[float]): array of parameters
        inputs (array[float]): 1-d input vector (data sample)

    Returns:
        array[float]: 1-d output vector in the form of [alpha0*<O0>, alpha1*<O1>]
    """
    
    # layer iteration
    for l in range(len(params[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
              qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=q)
    
    return [qml.expval(qml.Hermitian(dm_labels[i], wires=[i])) for i in range(n_qubits)]

In [ ]:
n_component = 6

X = tf.keras.Input(shape=(n_component,), name='Input_Layer')


# Quantum Layer
num_fc_layer = 5
q_fc_layer = qml.qnn.KerasLayer(q_fc, {"params": (2, num_fc_layer, n_component)}, output_dim=2)(X)

# Alpha Layer
alpha_layer = class_weights()(q_fc_layer)

model = tf.keras.Model(inputs=X, outputs=alpha_layer)

In [ ]:
# view each of the layer's name and its specifications
model.summary()

In [ ]:
model(X_train[0:3])

# set the initial weights to the params_fix that has been optimized layer by layer
# Note: please change the name of the keras layer accordingly based on the model.summary()
model.get_layer('keras_layer').set_weights([params_fix])

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(opt, loss='mse', metrics=["accuracy"])

In [ ]:
# Note: please put the correct directory to save the weights accordingly if you want to run the code
filepath = "./Model/Best6_layer5(layer_id=all)_set5000_10epoch_saved-model-{epoch:02d}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1,
                                                save_weights_only=True, save_best_only=False, mode='auto')

In [ ]:
H = model.fit(X_train, to_categorical(Y_train), epochs=10, batch_size=128, initial_epoch=0,
              validation_data=(X_test, to_categorical(Y_test)), verbose=1,
              callbacks=[checkpoint])

In [ ]:
# load the parameters that is considered giving the best balance between train and test accuracy
model.load_weights('./Model/Best6_layer5(layer_id=all)_set5000_10epoch_saved-model-08.hdf5')


# keep the parameters as the best params, also keep the weight vector alpha
best_params = model.get_weights()[0]
alpha = model.get_weights()[1]

**Testing the Performance**

In [ ]:
best_params

In [ ]:
alpha 

In [ ]:
# view each of the layer's name and its specifications
model.summary()

In [ ]:
# load the parameters to the model
# Note: please change the name of the keras layer accordingly based on the model.summary()
model.get_layer('keras_layer').set_weights([best_params])
model.get_layer('class_weights').set_weights([np.array([alpha])])

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
#AUC for training set

Y_pred_train = model.predict(X_train)
roc_auc_score(Y_train, Y_pred_train[:, 1])

In [ ]:
#AUC for testing set

Y_pred_test = model.predict(X_test)
roc_auc_score(Y_test, Y_pred_test[:, 1])